In [45]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine , text
import pandas as pd
import json

from datetime import datetime, timedelta

# This NK AWS credentials is for NK SG team internal use, do not share with partners
from nksg_team_jdbc_conn import AWS_ACCESS_KEY, AWS_SECRET_KEY


In [21]:
import geopandas
df_rwi = geopandas.read_file("Income level in Jakarta Metropolitan Area (Village or Kelurahan Level).geojson")
df_rwi.head()

,PROV_ID,PROVINCE,CITYREG_ID,CITY_REGION,DISTRICT_ID,DISTRICT,VILLAGE_ID,VILLAGE,RWI,CLASS,SPENDING,geometry
0,31,DKI JAKARTA,3174,KOTA JAKARTA BARAT,3174050,TAMBORA,3.174050e+09,DURI SELATAN,1.025996,High,1.200.000 - 6.000.000,"MULTIPOLYGON (((106.80800 -6.16138, 106.80800 ..."
1,31,DKI JAKARTA,3174,KOTA JAKARTA BARAT,3174050,TAMBORA,3.174050e+09,TANAH SEREAL,1.029112,High,1.200.000 - 6.000.000,"MULTIPOLYGON (((106.81172 -6.14811, 106.81182 ..."
2,31,DKI JAKARTA,3174,KOTA JAKARTA BARAT,3174050,TAMBORA,3.174050e+09,DURI UTARA,1.025996,High,1.200.000 - 6.000.000,"MULTIPOLYGON (((106.80810 -6.15641, 106.80664 ..."
3,31,DKI JAKARTA,3174,KOTA JAKARTA BARAT,3174050,TAMBORA,3.174050e+09,KRENDANG,1.025996,High,1.200.000 - 6.000.000,"MULTIPOLYGON (((106.80726 -6.14800, 106.80735 ..."
4,31,DKI JAKARTA,3174,KOTA JAKARTA BARAT,3174050,TAMBORA,3.174050e+09,JEMBATAN BESI,1.025996,High,1.200.000 - 6.000.000,"MULTIPOLYGON (((106.80121 -6.15401, 106.80122 ..."


In [101]:
df = pd.read_csv('RWI_with_tower_count.csv')
df.head()

,province,city_region,district,village,rwi,class,spending,tower_count,geometry
0,DKI JAKARTA,KOTA JAKARTA PUSAT,GAMBIR,DURI PULO,1.025996,High,1.200.000 - 6.000.000,442,"MULTIPOLYGON (((106.80143 -6.1580436, 106.8014..."
1,DKI JAKARTA,KOTA JAKARTA SELATAN,PASAR MINGGU,RAGUNAN,1.040410,High,1.200.000 - 6.000.000,1363,"MULTIPOLYGON (((106.8289589 -6.2794742, 106.82..."
2,DKI JAKARTA,KOTA JAKARTA SELATAN,KEBAYORAN BARU,RAWA BARAT,1.021368,High,1.200.000 - 6.000.000,416,"MULTIPOLYGON (((106.81463 -6.2329347, 106.8147..."
3,DKI JAKARTA,KOTA JAKARTA SELATAN,SETIA BUDI,KARET,1.600545,Very High,>6.000.000,903,"MULTIPOLYGON (((106.8304955 -6.2113739, 106.83..."
4,DKI JAKARTA,KOTA JAKARTA TIMUR,KRAMAT JATI,CILILITAN,1.012687,High,1.200.000 - 6.000.000,1017,"MULTIPOLYGON (((106.87168 -6.25599, 106.87177 ..."


In [102]:
df['village'] = df['village'].str.replace(' ', '', regex=True)

In [109]:
df_population = pd.read_csv('Population.csv')
df_population.head()

,PROVINSI,KABUPATEN ATAU KOTA,KECAMATAN,KODE DESA ATAU KELURAHAN,DESA ATAU KELURAHAN,JUMLAH PENDUDUK
0,JAWA BARAT,BOGOR,CIBINONG,3.201011e+09,PONDOK RAJEG,17037
1,JAWA BARAT,BOGOR,CIBINONG,3.201011e+09,KARADENAN,34175
2,JAWA BARAT,BOGOR,CIBINONG,3.201011e+09,HARAPANJAYA,27916
3,JAWA BARAT,BOGOR,CIBINONG,3.201011e+09,NANGGEWER,34347
4,JAWA BARAT,BOGOR,CIBINONG,3.201011e+09,NANGGEWER MEKAR,15909


In [110]:
df_population['DESA ATAU KELURAHAN'] = df_population['DESA ATAU KELURAHAN'].str.replace(' ', '', regex=True)

In [114]:
merged_df = df.merge(df_population, how='left', left_on='village', right_on='DESA ATAU KELURAHAN')
del merged_df['PROVINSI']
del merged_df['KABUPATEN ATAU KOTA']
del merged_df['KECAMATAN']
del merged_df['KODE DESA ATAU KELURAHAN']
del merged_df['DESA ATAU KELURAHAN']
merged_df.head()


,province,city_region,district,village,rwi,class,spending,tower_count,geometry,JUMLAH PENDUDUK
0,DKI JAKARTA,KOTA JAKARTA PUSAT,GAMBIR,DURIPULO,1.025996,High,1.200.000 - 6.000.000,442,"MULTIPOLYGON (((106.80143 -6.1580436, 106.8014...",25058.0
1,DKI JAKARTA,KOTA JAKARTA SELATAN,PASAR MINGGU,RAGUNAN,1.040410,High,1.200.000 - 6.000.000,1363,"MULTIPOLYGON (((106.8289589 -6.2794742, 106.82...",43728.0
2,DKI JAKARTA,KOTA JAKARTA SELATAN,KEBAYORAN BARU,RAWABARAT,1.021368,High,1.200.000 - 6.000.000,416,"MULTIPOLYGON (((106.81463 -6.2329347, 106.8147...",7369.0
3,DKI JAKARTA,KOTA JAKARTA SELATAN,SETIA BUDI,KARET,1.600545,Very High,>6.000.000,903,"MULTIPOLYGON (((106.8304955 -6.2113739, 106.83...",21210.0
4,DKI JAKARTA,KOTA JAKARTA SELATAN,SETIA BUDI,KARET,1.600545,Very High,>6.000.000,903,"MULTIPOLYGON (((106.8304955 -6.2113739, 106.83...",14033.0


In [115]:
merged_df.rename(columns={'JUMLAH PENDUDUK': 'population'}, inplace=True)

# Remove rows with NaN values in the 'population' column
#merged_df.dropna(subset=['population'], inplace=True)
merged_df['dim'] = merged_df['tower_count'] / merged_df['population']

In [117]:
merged_df.to_csv('main_visualization.csv', sep=',', index=False, encoding='utf-8')

In [118]:
merged_df.dropna(subset=['population'], inplace=True)

In [120]:
merged_df.to_csv('main3.csv', sep=',', index=False, encoding='utf-8')

In [94]:
numerical_columns = merged_df.select_dtypes(include=['number'])

# Create the correlation table
correlation_table = numerical_columns.corr()
correlation_table

,rwi,tower_count,population,dim
rwi,1.000000,0.394889,0.209153,0.288751
tower_count,0.394889,1.000000,0.382233,0.739862
population,0.209153,0.382233,1.000000,-0.086718
dim,0.288751,0.739862,-0.086718,1.000000


In [107]:
selected_columns = [ 'city_region', 'district','village']

# Create a mask to filter rows where 'population' is NaN
mask = merged_df['population'].isna()

# Use the mask to select rows with NaN population and the specified columns
result_df = merged_df.loc[mask, selected_columns]

In [74]:
result_df.to_csv('villages_with_missing_population.csv', header=['city_region', 'district','village'], index=False)

In [131]:
median_rwi = merged_df['rwi'].median()
df_lowerrwi = merged_df[merged_df['rwi'] < median_rwi].copy()
df_lowerrwi.head()

,province,city_region,district,village,rwi,class,spending,tower_count,geometry,population,dim
11,JAWA BARAT,BOGOR,TENJOLAYA,GUNUNGMULYA,0.113468,Very Low,<354.000,10,"MULTIPOLYGON (((106.7116435 -6.630693, 106.711...",7684.0,0.001301
12,JAWA BARAT,BOGOR,CIAWI,BOJONGMURNI,0.854500,High,1.200.000 - 6.000.000,6,"MULTIPOLYGON (((106.8930321 -6.6965123, 106.89...",5797.0,0.001035
13,JAWA BARAT,BOGOR,CIAWI,BANJARWARU,0.925790,High,1.200.000 - 6.000.000,43,"MULTIPOLYGON (((106.8572993 -6.66404, 106.8573...",10067.0,0.004271
14,JAWA BARAT,BOGOR,CISARUA,CISARUA,0.777443,Medium,354.000 - 532.000,181,"MULTIPOLYGON (((106.9393645 -6.6699935, 106.93...",10809.0,0.016745
15,JAWA BARAT,BOGOR,CISARUA,CISARUA,0.777443,Medium,354.000 - 532.000,181,"MULTIPOLYGON (((106.9393645 -6.6699935, 106.93...",9539.0,0.018975


In [132]:
median_rwi

0.9417334

In [135]:
df_lowerrwi['new'] = ((median_rwi - df_lowerrwi['rwi']) * 166.667).astype(int)
df_lowerrwi.head()

,province,city_region,district,village,rwi,class,spending,tower_count,geometry,population,dim,new
11,JAWA BARAT,BOGOR,TENJOLAYA,GUNUNGMULYA,0.113468,Very Low,<354.000,10,"MULTIPOLYGON (((106.7116435 -6.630693, 106.711...",7684.0,0.001301,138
12,JAWA BARAT,BOGOR,CIAWI,BOJONGMURNI,0.854500,High,1.200.000 - 6.000.000,6,"MULTIPOLYGON (((106.8930321 -6.6965123, 106.89...",5797.0,0.001035,14
13,JAWA BARAT,BOGOR,CIAWI,BANJARWARU,0.925790,High,1.200.000 - 6.000.000,43,"MULTIPOLYGON (((106.8572993 -6.66404, 106.8573...",10067.0,0.004271,2
14,JAWA BARAT,BOGOR,CISARUA,CISARUA,0.777443,Medium,354.000 - 532.000,181,"MULTIPOLYGON (((106.9393645 -6.6699935, 106.93...",10809.0,0.016745,27
15,JAWA BARAT,BOGOR,CISARUA,CISARUA,0.777443,Medium,354.000 - 532.000,181,"MULTIPOLYGON (((106.9393645 -6.6699935, 106.93...",9539.0,0.018975,27


In [136]:
df_lowerrwi = df_lowerrwi.sort_values(by='new', ascending=False)
df_lowerrwi.head()

,province,city_region,district,village,rwi,class,spending,tower_count,geometry,population,dim,new
324,JAWA BARAT,BOGOR,CIGUDEG,SUKAMAJU,-0.628857,Very Low,<354.000,3,"MULTIPOLYGON (((106.51879 -6.53667, 106.51924 ...",23272.0,0.000129,261
325,JAWA BARAT,BOGOR,CIGUDEG,SUKAMAJU,-0.628857,Very Low,<354.000,3,"MULTIPOLYGON (((106.51879 -6.53667, 106.51924 ...",9348.0,0.000321,261
329,JAWA BARAT,BOGOR,CIGUDEG,SUKAMAJU,-0.628857,Very Low,<354.000,3,"MULTIPOLYGON (((106.51879 -6.53667, 106.51924 ...",5652.0,0.000531,261
328,JAWA BARAT,BOGOR,CIGUDEG,SUKAMAJU,-0.628857,Very Low,<354.000,3,"MULTIPOLYGON (((106.51879 -6.53667, 106.51924 ...",68232.0,0.000044,261
327,JAWA BARAT,BOGOR,CIGUDEG,SUKAMAJU,-0.628857,Very Low,<354.000,3,"MULTIPOLYGON (((106.51879 -6.53667, 106.51924 ...",8088.0,0.000371,261


In [138]:
df_lowerrwi.to_csv('LowerRWI.csv', sep=',', index=False, encoding='utf-8')